#### Wednesday, Feburary 12, 2025

The example found at [Text-to-SQL](https://huggingface.co/docs/smolagents/examples/text_to_sql) has a Colab link which does not work, so I manually created this from [Text-to-SQL](https://github.com/huggingface/smolagents/blob/main/docs/source/en/examples/text_to_sql.md)

Playing with various models in this notebook demonstrates [Qwen/Qwen2.5-Coder-32B-Instruct](https://huggingface.co/Qwen/Qwen2.5-Coder-32B-Instruct) is a very good model.

ALL OF THE RUNS WORKED, SO DO NOT RE-RUN THEM! IF YOU WANT TO RUN THEM AGAIN, RUN THEM IN A NEW CELL!


In [1]:
# !pip install smolagents python-dotenv sqlalchemy --upgrade -q

In [2]:
from dotenv import load_dotenv
load_dotenv()

False

In [3]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [4]:
def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

In [5]:
table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

In [6]:
rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

In [7]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


In [8]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

In [ ]:
# For our local testing, we will use a local model
# ls ~/.cache/huggingface/transformers/

from smolagents import CodeAgent, TransformersModel

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" # wow! How does this model NOT fit on the 4090!??

model_id = "meta-llama/Llama-3.2-3B-Instruct" # This takes up 12642MiB of VRAM

model_id = "Qwen/Qwen2.5-Coder-14B-Instruct" # This downloads almost 30gb of files! ... no way this is gonna fit on the 4090!

model_id = "Qwen/Qwen2.5-Coder-7B-Instruct" # Wow! Even this model does not fit on the 4090! ... 14.5gb of files!


model = TransformersModel(model_id=model_id)

# 7m 50.9s to download ... 

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Failed to load tokenizer and model for model_id='Qwen/Qwen2.5-Coder-7B-Instruct': CUDA out of memory. Tried to allocate 260.00 MiB. GPU 0 has a total capacity of 23.55 GiB of which 22.06 MiB is free. Including non-PyTorch memory, this process has 23.50 GiB memory in use. Of the allocated memory 22.99 GiB is allocated by PyTorch, and 144.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables). Loading default tokenizer and model instead from default_model_id='HuggingFaceTB/SmolLM2-1.7B-Instruct'.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.03 GiB. GPU 0 has a total capacity of 23.55 GiB of which 22.06 MiB is free. Including non-PyTorch memory, this process has 23.50 GiB memory in use. Of the allocated memory 22.99 GiB is allocated by PyTorch, and 144.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct") # how about we use the same model specified below?
    # model = model
    # model=HfApiModel("meta-llama/Llama-3.2-3B-Instruct"),
    # model=HfApiModel("meta-llama/Meta-Llama-3.1-8B-Instruct"), # This model requires a PRO subscription ... 
)


In [ ]:
# DO NOT RUN THIS AGAIN! .... I WANT THESE RESULTS TO BE SAVED TO THIS NOTEBOOK! ... CUZ THIS RUN WORKED!

agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Find the maximum price in the receipts table                                                                   
  max_price_query = "SELECT MAX(price) FROM receipts"                                                              
  max_price = sql_engine(query=max_price_query)                                                                    
  print(f"The maximum price is: {max_price}")                                                                      
                                                                                                                   
  # Find the customer name with the maximum price                                                                  
  max_price_customer_query = f"SELECT customer_name FROM receipts WHERE price = {max_price}"                       
  max_price_customer = sql_engine(query=max_price_customer_query)                                                  
  print(f"The customer with the maximum price is: {max_price_customer}")                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The maximum price is: 
(53.43,)

Code execution failed at line 'max_price_customer = sql_engine(query=max_price_customer_query)' due to: 
OperationalError:(sqlite3.OperationalError) near ")": syntax error
[SQL: SELECT customer_name FROM receipts WHERE price = 
(53.43,)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Find the maximum price in the receipts table                                                                   
  max_price_query = "SELECT MAX(price) FROM receipts"                                                              
  max_price_result = sql_engine(query=max_price_query)                                                             
  max_price = float(max_price_result.strip().replace('(', '').replace(')', '').replace(',', ''))                   
  print(f"The maximum price is: {max_price}")                                                                      
                                                                                                                   
  # Find the customer name with the maximum price                                                                  
  max_price_customer_query = f"SELECT customer_name FROM receipts WHERE price = {max_price}"                       
  max_price_customer_result = sql_engine(query=max_price_customer_query)                                           
  max_price_customer = max_price_customer_result.strip().replace('(', '').replace(')', '').replace(',',            
  '').strip("'")                                                                                                   
  print(f"The customer with the maximum price is: {max_price_customer}")                                           
                                                                                                                   
  final_answer(max_price_customer)                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The maximum price is: 53.43
The customer with the maximum price is: Woodrow Wilson

Out - Final answer: Woodrow Wilson

[Step 3: Duration 25.50 seconds| Input tokens: 11,050 | Output tokens: 689]

'Woodrow Wilson'

#### Level 2: Table Joins

In [11]:
table_name = "waiters"
waiters = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
insert_rows_into_table(rows, waiters)

In [12]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


In [13]:
sql_engine.description = updated_description

agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct"), 
)

agent.run("Which waiter got more total money from tips?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which waiter got more total money from tips?                                                                    │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT w.waiter_name, SUM(r.tip) as total_tips                                                                   
  FROM waiters w                                                                                                   
  JOIN receipts r ON w.receipt_id = r.receipt_id                                                                   
  GROUP BY w.waiter_name                                                                                           
  """                                                                                                              
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Corey Johnson', 1.2)
('Margaret James', 1.0)
('Michael Watts', 5.67)

Out: None

[Step 0: Duration 7.94 seconds| Input tokens: 2,150 | Output tokens: 130]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Michael Watts")                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Michael Watts

[Step 1: Duration 4.39 seconds| Input tokens: 4,585 | Output tokens: 205]

'Michael Watts'